In [1]:
import pandas as pd
from datetime import datetime

In [23]:
files = [r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_2011-01-01_2013-01-01.csv',
         r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_2013-01-01_2016-01-01.csv',
         r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_2016-01-01_2016-06-30.csv',
         r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_2016-06-30_2017-06-30.csv',
         r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_2017-06-30_2018-06-30.csv',
            r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_2018-06-30_2020-01-01.csv',
            r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_2020-01-01_2021-01-01.csv',
            r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_2021-01-01_2023-01-01.csv',
            r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_2023-01-01_2024-05-26.csv',
            r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_error.csv',
            r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_over100.csv',
            r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions.csv']

df = pd.concat([pd.read_csv(f) for f in files])
df.to_csv(r'C:\Users\legion\Desktop\Reddit\repo\data\morocco\submissions_all.csv', index=False, encoding='utf-8-sig')

C:\Users\legion\AppData\Local\Temp\ipykernel_25980\383422846.py:14: DtypeWarning: Columns (3,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([pd.read_csv(f) for f in files])


## Submissions

In [63]:
subs = pd.read_csv('../data/morocco/submissions_all.csv', low_memory=False)

# drop unnecessary columns
subs.drop(columns=['Unnamed: 0', 'media', 'media_embed', 'thumbnail'], inplace=True)

# delete rows with [deleted by user] or [ Removed by Reddit ] in title
subs = subs[subs['title'] != '[deleted by user]']
subs = subs[subs['title'] != '[ Removed by Reddit ]']
subs = subs[subs['title'] != '[image processing failed]']

# replace [deleted] and [removed] with Empty str in selftext and author
subs['selftext'] = subs['selftext'].replace(['[deleted]', '[removed]', 'Title', 'title'], '')
subs['title'] = subs['title'].replace(['[deleted]', '[removed]', 'Title', 'title'], '')
subs['author'] = subs['author'].replace('[deleted]', '')

# replace nan with Empty str in all columns
subs.fillna('', inplace=True)

# drop duplicates
subs.drop_duplicates(subset=['id'], inplace=True)
subs.reset_index(drop=True, inplace=True)

# change created_utc to datetime type
subs['created_utc'] = pd.to_datetime(subs['created_utc'])

# order by created_utc
subs.sort_values(by='created_utc', inplace=True)


In [65]:
subs

,id,author,author_flair_text,title,selftext,link_flair_text,created_utc,permalink,score,num_comments,over_18,hide_score
88313,83vri,taoufix,,Facebook is lost case [pic],,None,2009-03-11 18:24:44,/r/Morocco/comments/83vri/facebook_is_lost_cas...,3,3,False,False
88314,c6u7c,,,Rabat Agdal At Night,,None,2010-05-21 21:43:14,/r/Morocco/comments/c6u7c/rabat_agdal_at_night/,3,2,False,False
88315,c7162,taoufix,,Beach near Sidi Ifni at sunset [pic],,None,2010-05-22 15:53:13,/r/Morocco/comments/c7162/beach_near_sidi_ifni...,4,0,False,False
88316,c71ir,,,"Medina de Rabat on a hazy, lazy friday",,None,2010-05-22 16:43:48,/r/Morocco/comments/c71ir/medina_de_rabat_on_a...,3,1,False,False
88317,c727d,taoufix,,Tiznit traditional market street during lunch ...,,None,2010-05-22 18:11:18,/r/Morocco/comments/c727d/tiznit_traditional_m...,3,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
85527,1d0ngog,Toji7707,:snoo_smile: Visitor,share with us your experiences in moroccan dat...,Feel free to post about:\n\n* Your own romanti...,:society: Society,2024-05-25 22:39:12,/r/Morocco/comments/1d0ngog/share_with_us_your...,1,1,False,
85526,1d0om6f,yello_penguin,:snoo_smile: Visitor,Pass piscine for one day,Does anyone know any good pools in rabat or ar...,:AskMorocco: AskMorocco,2024-05-25 23:38:38,/r/Morocco/comments/1d0om6f/pass_piscine_for_o...,1,1,False,
85525,1d0oq4b,karimelkh,:snoo_smile: Visitor,Finding internships in Morocco HELP,"Hi everyone,\n\nI am 1st yr CS student and I a...",:education: Education,2024-05-25 23:44:29,/r/Morocco/comments/1d0oq4b/finding_internship...,1,1,False,
85524,1d0ovqn,AdSoft2013,:snoo_smile: Visitor,getting a job with no experience,,:economy: Economy,2024-05-25 23:52:51,/r/Morocco/comments/1d0ovqn/getting_a_job_with...,1,1,False,


In [64]:
subs.to_csv('../data/cleaned/submissions.csv', index=False)

## Comments

In [3]:
comments = pd.read_csv('../data/morocco/comments.csv', low_memory=False)

# drop unnecessary columns
comments.drop(columns=['Unnamed: 0', 'subreddit_id', 'subreddit'], inplace=True)

# replace [deleted] and [removed] with Empty str in author
comments['author'] = comments['author'].replace('[deleted]', None)

# remove AutoModerator comments
comments = comments[comments['author'] != 'AutoModerator']

# drop rows with [deleted] or [removed] in body
comments = comments[comments['body'] != '[deleted]']
comments = comments[comments['body'] != '[removed]']

# drop duplicates
comments.drop_duplicates(subset=['id'], inplace=True)
comments.reset_index(drop=True, inplace=True)

# change created_utc to datetime type
comments['created_utc'] = pd.to_datetime(comments['created_utc'])

# format link_id and parent_id to match submission id
comments['link_id'] = comments['link_id'].str.split('_').str[1]
comments['parent_id'] = comments['parent_id'].str.split('_').str[1]


In [4]:
len(comments)

870662

In [13]:
comments.to_csv('../data/cleaned/comments.csv', index=False)